## 文件描述符

文件描述符是一个整数；  
所谓的文件描述符其实就是文件指针数组的Index，在这个文件指针数据中，前三位被填入默认值，分别指向标准输入流、标准输出流、标准错误流，分别是0，1，2

![img](img/file.jpg)

### 1. 日志
* **/var (/log)**下用于存放运行时需要改变数据的文件，也是某些大文件的溢出区，比方说**各种服务的日志文件**（系统启动日志等。）等；  
查看日志的命令和查看文件的命令是一样的：  


1. 查看日志常用命令  
    **tail:**  
       -n  是显示行号；相当于nl命令；例子如下：
            tail -100f test.log      实时监控100行日志
            tail  -n  10  test.log   查询日志尾部最后10行的日志;
            tail -n +10 test.log    查询10行之后的所有日志;

    **head:**  
        跟tail是相反的，tail是看后多少行日志；例子如下：

            head -n 10  test.log   查询日志文件中的头10行日志;

            head -n -10  test.log   查询日志文件除了最后10行的其他所有日志;

    **cat：** 
        tac是倒序查看，是cat单词反写；例子如下：

            cat -n test.log | grep "debug"   查询关键字的日志

 

2. 应用场景一：按行号查看---**过滤出关键字附近的日志**

     1）cat -n test.log |grep "debug"  得到关键日志的行号

     2）cat -n test.log |tail -n +92|head -n 20  选择关键字所在的中间一行. 然后查看这个关键字前10行和后10行的日志:

            tail -n +92表示查询92行之后的日志

            head -n 20 则表示在前面的查询结果里再查前20条记录

 

3. 应用场景二：**根据日期查询日志**

      sed -n '/2014-12-17 16:17:20/,/2014-12-17 16:17:36/p'  test.log

      特别说明:上面的两个日期必须是日志中打印出来的日志,否则无效；
        先 grep '2014-12-17 16:17:20' test.log 来确定日志中是否有该 时间点
        所选择要过滤的时间要和日志所用的时间一样

 

4. 应用场景三：日志内容特别多，打印在屏幕上不方便查看
    (1)使用more和less命令,

           如： cat -n test.log |grep "debug" |more     这样就分页打印了,通过点击空格键翻页

    (2)使用 >xxx.txt 将其保存到文件中,到时可以拉下这个文件分析

            如：cat -n test.log |grep "debug"  >debug.txt

### 2. 各个目录的含义


![](img/linux-file.png)

### 3. linux中查看一个文件中某个关键字出现的次数
* grep -o 'KeyWord' 03.txt | wc -l
* grep -o "keyWord" fileName | wc -l

### 4. 查看进程
* ps -l   列出与本次登录有关的进程信息；
* ps -f   查看进程信息，包括父进程ID
* ps -aux   查询内存中进程信息；
    * a：显示当前终端下的所有进程信息，包括其他用户的进程。

    * u：使用以用户为主的格式输出进程信息。

    * x：显示当前用户在所有终端下的进程。
* ps -aux | grep XXX   查询与XXX进程的详细信息；
* top   查看内存中进程的动态信息；
* kill -9 pid   杀死进程。

![](img/top.png)

### 5.创建进程

Linux使用**fork()**创建一个子进程，子进程在创建成功后，在子进程中返回 0，**从当前位置开始执行**

```c++
#include <unistd.h>

// 参数	： void 
// 返回值： pid_t 创建的子进程ID
pid_t fork(void);
```

返回值：fork() 返回值会返回**两次**，分别在**父进程和子进程**中返回。

1. **在父进程中返回子进程的ID，在子进程中返回0**。所以可以通过fork 的返回值来区分父进程与子进程。
2. 在父进程中返回 -1 ，表示创建子进程失败，并设置 errorno。如下面两种情况导致创建失败：

    * 当前系统的进程数已经达到了系统规定的上限，这时 errno 的值被设置为EAGAIN
    * 系统内存不足，这时 errno 的值被设置为 ENOMEM


在创建子进程时，Linux的fork 是通过 **写时拷贝 (copy-on-write)** 实现；在执行fork语句后，内核并不复制父进程的整个地址空间，而是父子进程共享父进程的地址空间（此时父子进程对于地址空间是**只读指令**），在父进程或者子进程进行**写**指令时，子进程才会复制一份地址空间，从而使得父子进程拥有自己的虚拟地址空间，在自己的地址空间进行写操作。

也就是说，**资源的复制是在需要写入时才会进行，在此之前，只会以只读方式进行共享。**

**对于文件资源**
* 如果在 **fork()之前打开了文件** ，则共享同一个文件偏移；
* 如果在 **fork()之后打开文件**，则双方有独立的文件偏移；

**查看linux最大进程数**  

* cat /proc/sys/fs/file-max　　查看系统级的最大限制
* ulimit -n　　查看用户级的限制